# M&M ASH
*Multivariate Bayesian variable selection via Adaptive Shrinkage, with applications to eQTL fine-mapping*

<hr>

## Meetings

[Here](doc/writeup/Meetings.html) is minutes for project meetings.

## Written documents 

### 2017/04
* [GTEx V7 analysis plan](doc/writeup/GTEx7_Analysis_Plan.html)

### 2017/03
* [A modular approach to m&m ash](doc/writeup/Modular_MNMASH.html)

### 2016/11
* [Estimating mixture proportions jointly for all genes in eQTL application](https://github.com/gaow/mvarbvs/tree/master/analysis/writeup/lr_diag_cov/dirichlet_ml)
* [m&m ash model assuming diagonal plus low rank covariance](https://github.com/gaow/mvarbvs/tree/master/analysis/writeup/lr_diag_cov)

### 2016/10
* [Start simple: a discussion with Peter Carbonetto](doc/writeup/Start_Simple.html)

### 2016/08
* [m&m ash model assuming identity covariance](https://github.com/gaow/mvarbvs/tree/master/analysis/writeup/identity_cov)
* [mr-ash model derivation](https://github.com/stephenslab/mvash/tree/add-latex/docs/derivations)